### Normal

In [ ]:
import code_generation
prompt_file_names = [
    "humaneval_java.jsonl",
]
using_Lora = False
model_name = "Salesforce/codegen-350M-mono"
tokenizer_name = "Salesforce/codegen-350M-mono"
generation_strategy = "ammarnasr/pass_at_10_gen_config"
gen_name = generation_strategy.split('/')[-1]
batch_size = 20
for prompt_file_name in prompt_file_names:
    model_name_short = model_name.split("/")[-1]
    model_name_short = model_name_short.replace("-", "_")
    prompt_file_name_short = prompt_file_name.split("/")[-1]
    prompt_file_name_short = prompt_file_name_short.split(".")[0]
    output_file_name = f"runs/{model_name_short}_{prompt_file_name_short}_{gen_name}_x.jsonl"

    wandb_project_name = f"{model_name_short}_{prompt_file_name_short}"

    args_dict = {
        "prompts_file_name": prompt_file_name,
        "model_name": model_name,
        "tokenizer_name": tokenizer_name,
        "generation_strategy": generation_strategy,
        "output_file_name": output_file_name,
        "wandb_project_name": wandb_project_name,
        "batch_size": batch_size,
    }
    print(args_dict)

    code_generation.main(args_dict, with_peft=using_Lora)



### Full

In [ ]:
import jsonlines as jsonl
import os
import json

prompt= '''
def intersection(interval1: Tuple[int, int], interval2: Tuple[int, int]) -> str:
    """
    Determines whether the intersection of two closed intervals results in a prime-length interval.
    
    Args:
        interval1 (Tuple[int, int]): The first closed interval as a tuple (start, end).
        interval2 (Tuple[int, int]): The second closed interval as a tuple (start, end).
        
    Returns:
        str: Returns 'YES' if the length of the intersection is a prime number, 'NO' otherwise.
            If the two intervals don't intersect, also returns 'NO'.
    """
'''
tests = '''
def check(candidate):
    assert candidate((1, 2), (2, 3)) == 'NO'
    assert candidate((-1, 1), (0, 4)) == 'NO'
    assert candidate((-3, -1), (-5, 5)) == 'YES'
    assert candidate((-2, 2), (-4, 0)) == 'YES'
    assert candidate((-11, 2), (-1, -1)) == 'NO'
    assert candidate((1, 2), (3, 5)) == 'NO'
    assert candidate((1, 2), (1, 2)) == 'NO'
    assert candidate((-2, -2), (-3, -2)) == 'NO'

def test_check():
    check(intersection)

test_check()

'''


#File name 
file_name = "custome_py.jsonl"

#Open file
with jsonl.open(file_name) as reader:
    for obj in reader:
        print(obj)
        break

#Update file
obj['prompt'] = prompt
obj['tests'] = tests
obj['name'] = "my_prompt"

#Write to file
with jsonl.open(file_name, mode='w') as writer:
    writer.write(obj)

dir_name = "custome_runs"
if not os.path.exists(dir_name):
    os.mkdir(dir_name)


#Open file
with jsonl.open(file_name) as reader:
    for obj in reader:
        # print(obj['prompt'])
        break



In [ ]:
def generate_results(exp_name):
    convert_to_pre_eval(exp_name)
    run_evaluation(exp_name)
def convert_to_pre_eval(exp_name):
    sf = f'custome_runs/{exp_name}'
    td = f'custome_runs/tgt/{exp_name.replace("-", "")}'
    !python convert_to_pre_eval.py --source_file $sf --target_dir $td
    print(f'Converted {sf} to {td}')
def run_evaluation(exp_name):
    sf = f'custome_runs/{exp_name}'
    td = f'custome_runs/tgt/{exp_name.replace("-", "")}'
    eval_cmd = f"podman run --rm --network none -v ./{td}:/{td}:rw multipl-e-eval --dir /{td} --output-dir /{td} --recursive"
    !{eval_cmd}


def get_results(results_file):
    with open(results_file, 'r') as f:
        results = json.load(f)
    all_results = []
    for r in results['results']:
        # print(r['status'])
        all_results.append(r['status'])
    return all_results

In [ ]:
import code_generation
prompt_file_names = [
    "custome_py.jsonl",
]
k = '100'
using_Lora = False
model_name = "Salesforce/codegen-350M-mono"
tokenizer_name = "Salesforce/codegen-350M-mono"
generation_strategy = f"ammarnasr/pass_at_{k}_gen_config"
gen_name = generation_strategy.split('/')[-1]
batch_size = 5
for prompt_file_name in prompt_file_names:
    model_name_short = model_name.split("/")[-1]
    model_name_short = model_name_short.replace("-", "_")
    prompt_file_name_short = prompt_file_name.split("/")[-1]
    prompt_file_name_short = prompt_file_name_short.split(".")[0]
    output_file_name = f"custome_runs/{model_name_short}_{prompt_file_name_short}_{gen_name}.jsonl"

    wandb_project_name = f"{model_name_short}_{prompt_file_name_short}"

    args_dict = {
        "prompts_file_name": prompt_file_name,
        "model_name": model_name,
        "tokenizer_name": tokenizer_name,
        "generation_strategy": generation_strategy,
        "output_file_name": output_file_name,
        "wandb_project_name": wandb_project_name,
        "batch_size": batch_size,
    }
    print(args_dict)

    code_generation.main(args_dict, with_peft=using_Lora)



In [ ]:
import numpy as np
import json
exp_name = f'codegen_350M_mono_custome_py_pass_at_{k}_gen_config.jsonl'
results_file = f'custome_runs/tgt/codegen_350M_mono_custome_py_pass_at_{k}_gen_config.jsonl/my_prompt.results.json'

generate_results(exp_name)
all_results = get_results(results_file)

In [ ]:

all_results = np.array(all_results)
print(np.unique(all_results, return_counts=True))